# 创建部署

让我们创建一个在模块 5 中创建的 `task_maistro` 应用程序的部署。

## 代码结构

[应提供以下信息](https://docs.langchain.com/langsmith/application-structure)以创建 LangGraph Platform 部署：

* 一个 [LangGraph API 配置文件](https://docs.langchain.com/langsmith/application-structure#configuration-file) - `langgraph.json`
* 实现应用程序逻辑的图 - 例如，`task_maistro.py`
* 一个指定运行应用程序所需依赖项的文件 - `requirements.txt`
* 提供应用程序运行所需的环境变量 - `.env` 或 `docker-compose.yml`

我们已经在 `module-6/deployment` 目录中拥有这些内容！

## CLI

[LangGraph CLI](https://docs.langchain.com/langsmith/cli) 是用于创建 LangGraph Platform 部署的命令行界面。

In [1]:
%%capture --no-stderr
%pip install -U langgraph-cli

要创建一个 <!-- [~self-hosted deployment~](https://langchain-ai.github.io/langgraph/how-tos/deploy-self-hosted/#how-to-do-a-self-hosted-deployment-of-langgraph) --> [自托管部署](https://docs.langchain.com/langsmith/self_hosted_data_plane)，我们将遵循几个步骤。

### 为 LangGraph Server 构建 Docker 镜像

我们首先使用 langgraph CLI 为 [LangGraph Server](https://docs.google.com/presentation/d/18MwIaNR2m4Oba6roK_2VQcBE_8Jq_SI7VHTXJdl7raU/edit#slide=id.g313fb160676_0_32) 创建一个 Docker 镜像。

这将把我们的图和依赖项打包到一个 Docker 镜像中。

Docker 镜像是 Docker 容器的模板，包含运行应用程序所需的代码和依赖项。

确保 [Docker](https://docs.docker.com/engine/install/) 已安装，然后运行以下命令创建 Docker 镜像 `my-image`：

```
$ cd module-6/deployment
$ langgraph build -t my-image
```

### 设置 Redis 和 PostgreSQL

如果你已经有 Redis 和 PostgreSQL 在运行（例如，在本地或其他服务器上），那么可以使用 Redis 和 PostgreSQL 的 URI [单独](https://docs.langchain.com/langsmith/deploy-hybrid#running-the-application-locally)创建并运行 LangGraph Server 容器：

```
docker run \
    --env-file .env \
    -p 8123:8000 \
    -e REDIS_URI="foo" \
    -e DATABASE_URI="bar" \
    -e LANGSMITH_API_KEY="baz" \
    my-image
```

或者，你可以使用提供的 `docker-compose.yml` 文件根据定义的服务创建三个独立的容器：

* `langgraph-redis`: 使用官方 Redis 镜像创建新容器。
* `langgraph-postgres`: 使用官方 Postgres 镜像创建新容器。
* `langgraph-api`: 使用你预构建的镜像创建新容器。

只需复制 `docker-compose-example.yml` 并添加以下环境变量来运行部署的 `task_maistro` 应用程序：

* `IMAGE_NAME`（例如，`my-image`）
* `LANGSMITH_API_KEY`
* `OPENAI_API_KEY`

然后，<!-- [~launch the deployment~](https://langchain-ai.github.io/langgraph/how-tos/deploy-self-hosted/#using-docker-compose) [launch the deployment](https://docs.langchain.com/langsmith/self_hosted_data_plane): --> 启动部署！

```
$ cd module-6/deployment
$ docker compose up
```